In this notebook we do the annealing for the adaptive schedule derived from the fluctuation analysis done in "analyse_fluctuations_k.ipynb"

**Just check what fluctuation and metric it is using.**

In [ ]:
using AdaptiveQuantumAnnealing

In [ ]:
# The objective of this notebook is to merge the adaptive part with the AGP
using  SpinFluctuations
using LinearAlgebra, PyPlot, Printf, DelimitedFiles
using Interpolations, DifferentialEquations
using Symbolics, ForwardDiff
using PyCall, DataFrames
using HDF5, QAOA

# include("/home/ubuntu/Archives/projects/AdaptiveQuantumAnnealing.jl/src/utils.jl")
# include("/home/ubuntu/Archives/projects/AdaptiveQuantumAnnealing.jl/src/adiabatic_gauge_potential.jl")
include("/home/ubuntu/.julia/dev/AdaptiveQuantumAnnealing/src/qutip_utils.jl")

qp = pyimport("qutip")
np = pyimport("numpy")
PyPlot.plt.style.use("~/paper.mplstyle") 
rc("text", usetex=true)

### Open file

In [ ]:
# # # Extra.csv: Mirkarimi raw data
# # # If you are running this, keep in mind you do not have the exact arpack stored in lambda above, but nonetheless we redo this calculation below
# # # ms_filename =  "khdzusjsapqkkrbpjlknwsbxdbijax"
# # ms_filename =  "khdzusjsapqkkrbpjlknwsbxdbijax"
# # ms_folder_path = "/home/ubuntu/Archives/data/MAX2SATQuantumData/Mirkarimi_ComparingHardness/instances_typical/"
# ms_folder_path = "/home/ubuntu/Archives/data/MAX2SAT/crosson/"
# ms_filename = string(0)
# ms_file_path =  ms_folder_path * ms_filename * ".m2s"
# N = 20

In [ ]:
# # To get the J and h for the mirkarimi raw
# types_to_values = Dict("type_1" => (1, 1, 1), "type_2" => (1, -1, -1), "type_3" => (-1, 1, -1), "type_4" => (-1, -1, 1))
# function get_clause_type_from(pair_of_unary_logical_connectives)
#     mapping = Dict((1, 1) => "type_1", (1, -1) => "type_2", (-1, 1) => "type_3", (-1, -1) => "type_4")
#     mapping[pair_of_unary_logical_connectives]
# end

# rawdata = readdlm(ms_file_path)

# not_negated_1 = rawdata[:, 1]
# not_negated_2 = rawdata[:, 3]
# unary_logical_connectives = zip(not_negated_1, not_negated_2) |> collect 
# clause_types = get_clause_type_from.(unary_logical_connectives)

# vars_1 = (rawdata[:, 2] .|> Int) .+ 1
# vars_2 = (rawdata[:, 4] .|> Int) .+ 1    
# clause_vars = zip(vars_1, vars_2) .|> collect

# clause_types = get_clause_type_from.(unary_logical_connectives)

# C = zip(clause_vars, clause_types)  |> collect

# h = zeros(N)
# J = zeros(N, N)
# for c in C
#     h[c[1][1]] += -types_to_values[c[2]][1]
#     h[c[1][2]] += -types_to_values[c[2]][2]
#     J[c[1][1], c[1][2]] += -types_to_values[c[2]][3]
# end
# J_mat = J + transpose(J);

# mf_problem = Problem(0, h, J_mat);

# gap_loc = 0.0;

In [ ]:

# # TIM SK 8
# J_mat = [0.0000 -0.3935 -0.0293 -0.1723 0.2925 0.2980 -0.0653 0.0795 0.3231; -0.3935 0.0000 0.2155 -0.1747 0.0003 -0.1460 0.3691 -0.0424 0.7848; -0.0293 0.2155 0.0000 -0.0278 0.0775 -0.0089 0.4674 0.5540 -0.1661; -0.1723 -0.1747 -0.0278 0.0000 -0.4918 -0.2860 0.7536 -0.5298 -0.2358; 0.2925 0.0003 0.0775 -0.4918 0.0000 0.4682 -0.0260 -0.0048 -0.1009; 0.2980 -0.1460 -0.0089 -0.2860 0.4682 0.0000 0.0612 0.3231 0.1912; -0.0653 0.3691 0.4674 0.7536 -0.0260 0.0612 0.0000 -0.1662 -0.2366; 0.0795 -0.0424 0.5540 -0.5298 -0.0048 0.3231 -0.1662 0.0000 -0.2560; 0.3231 0.7848 -0.1661 -0.2358 -0.1009 0.1912 -0.2366 -0.2560 0.0000]
# mf_problem = Problem(2^12,J_mat);

# gap_loc = 0.7878787878787878 # derived from another notebook, nothing comlex

In [ ]:
a = "/home/ubuntu/.julia/dev/AdaptiveQuantumAnnealing/data/instances/random_SK_instance_N_9_seed_131412.h5"

J_mat  = h5read(a,"J")
mf_problem = Problem(2^12,J_mat);
gap_loc = 0.7878787878787878

In [ ]:
# using CSV, Tables
# CSV.write("J_mat.csv", Tables.table(J_mat))
# J_mat

In [ ]:
# system_size = 11
# file_path = @sprintf("/home/ubuntu/Archives/data/tb_mount/N_%s/small_gaps/", string(system_size))

# # results_file = @sprintf("results_random_SK_instance_N_%s_seed_", string(system_size)) * string(seed) * ".h5" 
# seed = 2809
# instance_file = @sprintf("random_SK_instance_N_%s_seed_", string(system_size)) * string(seed) * ".h5";

# J_mat  = h5read(file_path * instance_file,"J")
# mf_problem = Problem(0,J_mat); # This is just to read the problem
# gap_loc = 0.8181818181818182

In [ ]:
J_prime = mf_problem.couplings
h_prime = mf_problem.local_fields
N = mf_problem.num_qubits;

### Fluctuations

In [ ]:
# npts = 8192
npts = 2048
T_final = 30000.
tol = 1e-8;


coarse_times = range(0, 1, npts + 1);
lyapunov_parameters = LyapunovParameters(T_final, npts, tol, tol);

In [ ]:
# This gives the action matrix(20X20) for the gaussian fluctuation - 2048 points 
sol, stat_GF = statistical_green_function(mf_problem, lyapunov_parameters);

In [ ]:
# Refer to the eq. 11.20 from Altman, Retard green functions 
flucs = k -> (real.(1.0im .* diag(stat_GF[k])[1:mf_problem.num_qubits]) .- 1.0) ./ 2;
all_flucs = reduce(hcat, map(flucs, 1:npts+1));

In [ ]:
# Set figure size
figure(figsize=(10, 4))

# Define the y-axis range and tick positions
ylims = (0, 0.15)
ytick_positions = [0, 0.075, 0.15]  # Manually specify the tick positions

# Plotting loop
for i in 1:(N ÷ 2)
    ax = subplot(2, (N ÷ 2), i)
    c = "k"
    plot(coarse_times, smoothen(all_flucs[i, :], coarse_times), c=c, label = "Spin $i", linewidth=1.5)
    axvline(gap_loc, ls="--", color="C0", label = "Gap loc.")
    xlim(0., 1)
    ax.set_xticklabels([])
    yticks(ytick_positions)  # Set y-axis ticks
    if i > 1
        ax.set_yticklabels([])
    end
    if i == 2
        legend(loc="upper left", fontsize=11)
    end
end

# Plotting loop for the second row
for i in ((N ÷ 2) + 1):N
    ax = subplot(2, (N ÷ 2), i)
    c = "k"
    plot(coarse_times, smoothen(all_flucs[i, :], coarse_times), c=c, label = "Spin $i", linewidth=1.5)
    axvline(gap_loc, ls="--", color="C0", label = "Gap loc.")
    xlim(0., 1)
    yticks(ytick_positions)  # Set y-axis ticks
    xlabel(L"$s$", fontsize=14)
    if i > ((N ÷ 2) + 1)
        ax.set_yticklabels([])
    else
        ax.set_ylabel(L"$F_{ii}(t, t)$", fontsize=14)
    end    
    if i == 5 || i == 7
        legend(loc="upper left", fontsize=11)
        
    end
end

# Tight layout
tight_layout()



# Display figure
display(gcf())
# savefig("/home/ubuntu/Archives/projects/Master_thesis/graphics/results/sk_instance_2/fluctuations.pdf")


# Feed the top index from trajectory analysis

In [ ]:
# top_idx = 5

In [ ]:
cut_solution(x) = sol.(x .* sol.t[end]); # Just converts the domain from [0,30k]->[0,1]
nx_coarse = hcat([matrix[1, :] for matrix in cut_solution(coarse_times)]...); # Extracting the individual spin from the sol
ny_coarse = hcat([matrix[2, :] for matrix in cut_solution(coarse_times)]...);
nz_coarse = hcat([matrix[3, :] for matrix in cut_solution(coarse_times)]...);

scaled_flucs(scale_factor, spin_nr) = smoothen(scale_factor.^2 .* all_flucs[spin_nr, :], coarse_times, navg=128) # gives the scaled flucs.

In [ ]:
# Beautification
# Set figure size
figure(figsize=(4, 4))

# Calculate scale factor
scale_factor = 1 .+ abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse)).^2

# Plot scaled fluctuations
subplot(212)
plot(coarse_times, scaled_flucs(scale_factor, top_idx))
axvline(gap_loc, ls="--", c="k", alpha=0.5)
xlim(0, 1)
ylim(0, )  # Adjust ylim as needed
xlabel("\$s\$")
ylabel("\$F_{ii}\$ (Scaled)")
title("\$i = 2\$", fontsize=14)  # Set subplot title with reduced font size

# Tight layout
tight_layout()

# Display figure
display(gcf())


In [ ]:
# Gaussian with a peak in at the gap location
function generate_gaussian(peak_position)
    σ = 0.05  # SD- adjust
    # A = 1 / exp(-0.5 * ((0.5 - peak_position) / σ)^2)  # Amplitude to ensure f(0) = f(1) = 0 # Not sure wether if this does that 

    gaussian(x) = 4 * exp(-0.5 * ((x - peak_position) / σ)^2)
    return gaussian
end

In [ ]:
Chi_new_metric = scaled_flucs(scale_factor, top_idx);
# Chi_new_metric_norm = Chi_new_metric ./ maximum(Chi_new_metric);
Chi_new_metric_norm = generate_gaussian(argmax(Chi_new_metric) / length(coarse_times)).(coarse_times); # gives a gaussian
# Chi_new_metric_norm = Chi_new_metric_norm ./ maximum(Chi_new_metric_norm);# This should normalise it to max height one??
Chi_new_metric = copy(Chi_new_metric_norm);

In [ ]:
# Set figure size
figure(figsize=(10, 3))

# Calculate scale factor
scale_factor = 1 .+ abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse)).^2

# Plot scaled fluctuations on the left
subplot(1, 2, 1)
plot(coarse_times, scaled_flucs(scale_factor, top_idx))
axvline(gap_loc, ls="--", c="k", alpha=0.5, label="Gap Position")
xlim(0, 1)
ylim(0, )  # Adjust ylim as needed
xlabel("\$s\$")
ylabel("\$F_{ii}\$ (Scaled)")
title("\$i = 2\$", fontsize=10)  # Set subplot title with reduced font size
legend() 

# Plot Chi_new_metric_norm on the right
subplot(1, 2, 2)
plot(coarse_times, Chi_new_metric_norm)
axvline(gap_loc, ls="--", c="k", alpha=0.5)
xlim(0, 1)
ylim(0, )  # Adjust ylim as needed
xlabel("\$s\$")
# ylabel("Metric")
title("Derived metric", fontsize=10)  # Set subplot title with reduced font size

# Tight layout
tight_layout()

# Display figure
display(gcf())
# savefig("/home/ubuntu/Archives/projects/Master_thesis/graphics/results/sk_instance_2/schedule.pdf")

In [ ]:
figure(figsize = (4,3))

plot(coarse_times, scaled_flucs(scale_factor, top_idx), label = "Fluctuation(i=2)")
plot(coarse_times, Chi_new_metric, label = " Gaussian peak")
axvline(argmax(Chi_new_metric) / length(coarse_times), linestyle = "--", color = "grey", label = "Peak loc.")
legend()
xlim(0, 1)
ylim(0, 1)  # Adjust ylim as needed
xlabel("\$s(t)\$")
# ylabel("\$F_{ii}\$ (Scaled)")

display(gcf())
# savefig("/home/ubuntu/Archives/projects/Master_thesis/graphics/results/sk_instance_2/gaussian_metric.pdf")

In [ ]:
T_f_in = 2^10
taulist = range(0, T_f_in, trunc(Int, 10* T_f_in)) # for the annealing

In [ ]:
g_new = linear_interpolation(coarse_times, Chi_new_metric)
g_feed = x-> 1 * g_new(x); # This multiplication with a scalar is to enhance the flatness at the gap region

In [ ]:
adaptive_schedule_parameters = AdaptiveScheduleParameters(taulist[end], RK4(), [0.0, 1.], 10000, 20, 1e-7, 1e-9)
adaptive_schedule = solve_boundary_value_problem(g_feed, adaptive_schedule_parameters);

In [ ]:

figure(figsize=(4, 3))

# Plot the schedule
plot(taulist, adaptive_schedule.(taulist), label="A=4", linewidth=1.5)

# Add a vertical line at the gap location
axvline(gap_loc * taulist[end], ls="--", c="gray", alpha=0.7, label="Gap loc.")

# Set limits and labels
xlim(taulist[1], taulist[end])
xlabel("Time (\$t\$)")
ylabel("\$s(t)\$")
ylim(0, 1)

# Add legend with improved formatting
legend(frameon=false)#, loc = "upper left")

# Add gridlines for better readability
grid(alpha=0.3)

# Tight layout
tight_layout()

# Display figure
display(gcf())

# savefig("/home/ubuntu/Archives/projects/Master_thesis/graphics/results/sk_instance_2/schedule_3.pdf")


### AGP part
Now that we have the schedule, we have to modify the action function from the AGP for this.

In [ ]:
# @variables J[1:N,1:N] h_d[1:N] h_p[1:N]
h_p = -h_prime;
h_d = -ones(N);
J = -copy(J_prime); # The minus sign is purely because of the requirement by the Action function. I missed a minus sign in the derivation and that is what is causing all this confusion 


In [ ]:
pauli_string = AdaptiveQuantumAnnealing.pauli_string_struct(pauli_string_generator(N)...); # This is the pauli strings

init_quantum_states = init_quantum_ground_state(N);
H0, H1 = quantum_hamiltonians(N, -ones(N), -h_p, -J); # The problem parameters that need to go into this has to be the actual J and the h_p vector. So I add a minus sign before to nullify the minus

In [ ]:
@variables alpha[1:N] beta[1:N,1:N] gamma[1:N,1:N] s
alpha_function_interp, beta_function_interp, gamma_function_interp = agp_coeffs(J, h_d, h_p, alpha, beta, gamma, s, 20000);

### For now let us do sesolve.

In [ ]:
linear_schedule(x) = x/taulist[end]

In [ ]:
f_prime = y->ForwardDiff.derivative(adaptive_schedule,y)

# # Adaptive sesolve

h_t_alpha_adaptive = [[pauli_string.sy_list[i], (t,_) -> (N * alpha_function_interp[i](adaptive_schedule(t))) * f_prime(t)] for i in 1:N];
h_t_beta_adaptive = [[(pauli_string.sx_list[i] * pauli_string.sy_list[j] + pauli_string.sx_list[j] * pauli_string.sy_list[i]), (t,_) -> (beta_function_interp[i,j](adaptive_schedule(t))) * f_prime(t)/(2)] for i in 1:N for j in 1:N];
h_t_gamma_adaptive = [[pauli_string.sz_list[i] * pauli_string.sy_list[j] + pauli_string.sz_list[j] * pauli_string.sy_list[i], (t,_) -> (gamma_function_interp[i,j](adaptive_schedule(t)) )* f_prime(t)/(2)] for i in 1:N for j in 1:N];
h_t_og_adaptive = [[H0, (t,_) -> (1 - adaptive_schedule(t))], [H1, (t,_) ->  (adaptive_schedule(t))]] ; # The regular adiabatic one

# An alternative method to the above cell
h_t_adaptive = copy(h_t_og_adaptive)
h_t_adaptive = copy(vcat(h_t_adaptive, h_t_alpha_adaptive))
h_t_adaptive = vcat(h_t_adaptive, h_t_beta_adaptive)
h_t_adaptive = vcat(h_t_adaptive, h_t_gamma_adaptive);

In [ ]:
# AGP vs No AGP
sol_evolve_adaptive = qp.sesolve(h_t_og_adaptive, init_quantum_states, taulist) #, e_ops= [sx_list[spin_nr], sy_list[spin_nr], sz_list[spin_nr]])
adaptive_probs = [abs(i[1]^2) for i in sol_evolve_adaptive.states[end]];


In [ ]:

sol_evolve_adaptive_agp = qp.sesolve(h_t_adaptive, init_quantum_states, taulist)#, options = options) #, e_ops= [sx_list[spin_nr], sy_list[spin_nr], sz_list[spin_nr]])
adaptive_agp_probs = [abs(i[1]^2) for i in sol_evolve_adaptive_agp.states[end]];

In [ ]:
# # Linear sesolve
h_t_alpha_linear = [[pauli_string.sy_list[i], (t,_) -> (N * alpha_function_interp[i](t/taulist[end]))/taulist[end]] for i in 1:N];
h_t_beta_linear = [[(pauli_string.sx_list[i] * pauli_string.sy_list[j] + pauli_string.sx_list[j] * pauli_string.sy_list[i]), (t,_) -> (beta_function_interp[i,j](t/taulist[end]))/(2 * taulist[end])] for i in 1:N for j in 1:N];
h_t_gamma_linear = [[pauli_string.sz_list[i] * pauli_string.sy_list[j] + pauli_string.sz_list[j] * pauli_string.sy_list[i], (t,_) -> (gamma_function_interp[i,j](t/taulist[end]))/(2 * taulist[end])] for i in 1:N for j in 1:N];
h_t_og_linear = [[H0, (t,_) -> ((taulist[end] - t) / taulist[end])], [H1, (t,_) ->  (t / taulist[end])]] ; # The regular adiabatic one
# An alternative method to the above cell
h_t_linear = copy(h_t_og_linear)
h_t_linear = copy(vcat(h_t_linear, h_t_alpha_linear))
h_t_linear = vcat(h_t_linear, h_t_beta_linear)
h_t_linear = vcat(h_t_linear, h_t_gamma_linear);

In [ ]:
sol_evolve_linear_agp = qp.sesolve(h_t_linear, init_quantum_states, taulist)
linear_agp_probs = [abs(i[1]^2) for i in sol_evolve_linear_agp.states[end]];



In [ ]:

sol_evolve_linear = qp.sesolve(h_t_og_linear, init_quantum_states, taulist) #, e_ops= [sx_list[spin_nr], sy_list[spin_nr], sz_list[spin_nr]])
linear_probs = [abs(i[1]^2) for i in sol_evolve_linear.states[end]];

In [ ]:
# sort(AdaptiveQuantumAnnealing.energies_and_bitstrings_df_sesolve(mf_problem), [:energy])

In [ ]:
df_sesolve = AdaptiveQuantumAnnealing.energies_and_bitstrings_df_sesolve(mf_problem)
df_sesolve[!,"linear_probs"] = linear_probs;
df_sesolve[!,"linear_agp_probs"] = linear_agp_probs;
df_sesolve[!,"adaptive_probs"] = adaptive_probs;
df_sesolve[!,"adaptive_agp_probs"] = adaptive_agp_probs
df_sesolve[!, "Row_number"] = 1:2^N # you can find the probability to look out for in the solution given out by energies_and_bitstrings_df_sesolve

sort!(df_sesolve, [:energy]);



In [ ]:
# using Arrow
# Arrow.write(save_adaptive_path * "/" * string(trunc(Int,T_f_in)) *"_8_.arrow", df)

In [ ]:
df
# There is definitley something strange going on with 10k points for interpolation, every other entry works like a charm. But just 10k fails # I tried restaring the notebook and running from scratch. But 10k???????????????


In [ ]:
# sum(df[!,"adaptive_probs"])

In [ ]:
maximum(adaptive_agp_probs)
sol_evolve_linear.states[1]
# Have to encode the actual solution in Qutip
init_quantum_states
abs(init_quantum_states.overlap(sol_evolve_linear.states[1]))

In [ ]:
function fidelity_function(O)
    fidelity_vector = []#Array{Float64}(undef,256)
    for temp_sol in sol_evolve_adaptive.states
        push!(fidelity_vector,abs(temp_sol[O][1] ^ 2));
    end
    return fidelity_vector
end

init_fidelity = [abs(init_quantum_states.overlap(temp_sol))^2 for temp_sol in sol_evolve_adaptive.states];

In [ ]:
using JLD2
# save_object("/home/ubuntu/.julia/dev/AdaptiveQuantumAnnealing/notebooks/sk_overlap_fidelity_data/gs_adap.jld2",fidelity_function(17))
# save_object("/home/ubuntu/.julia/dev/AdaptiveQuantumAnnealing/notebooks/sk_overlap_fidelity_data/1ex_adap.jld2",fidelity_function(48))
# save_object("/home/ubuntu/.julia/dev/AdaptiveQuantumAnnealing/notebooks/sk_overlap_fidelity_data/init_state_adap.jld2",init_fidelity);


In [ ]:
figure(figsize = (4,3))

plot(taulist, fidelity_function(17), linestyle = "-", label=L"Solution\, fidelity", color="C0")
plot(taulist, fidelity_function(48), linestyle = "-", label=L"1^{st}\, excited", color="C1")
plot(taulist, init_fidelity, linestyle = "-", label=L"Initial\, state", color="C2")

xlabel(L"T_{\mathrm{Final}}")
ylabel("Overlap")
legend(loc = 6,fontsize = 11)
ylim(0,1)
xlim(0,)
tight_layout()
display(gcf())

# # savefig("/home/ubuntu/Archives/projects/Master_thesis/graphics/results/sk_instance_2/fidelity_adaptive.pdf")

In [ ]:


# Beautification
figure(figsize=(4, 3))

# Data

fidelity_17 = fidelity_function(17)
fidelity_48 = fidelity_function(48)
init_fidelity = init_fidelity  # Example data for init_fidelity

# Plot data
plot(taulist, fidelity_17, linestyle="-", label=L"Solution\, fidelity", color="C0")
plot(taulist, fidelity_48, linestyle="-", label=L"1^{st}\, excited", color="C1")
plot(taulist, init_fidelity, linestyle="-", label=L"Initial\, state", color="C2")

# Labels and Legend
xlabel(L"T_{\mathrm{Final}}")
ylabel("Overlap")
legend(loc="lower left", fontsize=11)
yticks([0.0, 0.5, 1.0])
# Adjustments
ylim(0, 1)
xlim(0, maximum(taulist))
tight_layout()

# Display figure
display(gcf())
# savefig("/home/ubuntu/Archives/projects/Master_thesis/graphics/results/sk_instance_2/fidelity_linear.pdf")

### Shift things to QAOA (works better for larger systems)

In [ ]:
T_f_in

In [ ]:

mf_problem = Problem(trunc(Int, T_f_in * 20),h, J_mat);

linear_schedule(x) = x/taulist[end]
f_prime = y->ForwardDiff.derivative(adaptive_schedule,y)


In [ ]:

adaptive_probs = anneal(mf_problem, x->round((adaptive_schedule(x)),digits = 5), taulist[end]);
# adaptive_agp_probs = anneal_CD(mf_problem, x->adaptive_schedule(x), taulist[end], alpha_function_interp, beta_function_interp, gamma_function_interp)
;

In [ ]:

# linear_agp_probs = anneal_CD(mf_problem, linear_schedule, taulist[end], alpha_function_interp, beta_function_interp, gamma_function_interp)
linear_probs = anneal(mf_problem, linear_schedule, taulist[end]);

In [ ]:
# agp_probs_qaoa = anneal_CD(mf_problem, x->adaptive_schedule(x), taulist[end], alpha_function_interp, beta_function_interp, gamma_function_interp)
# # agp_probs_qaoa = anneal_CD(mf_problem, x->adaptive_schedule(x), taulist[end], alpha_function_interp, beta_function_interp, gamma_function_interp)
# no_agp_probs_qaoa = anneal(mf_problem, x->round((adaptive_schedule(x)),digits = 5), taulist[end])
# linear_vanila_probs_qaoa = anneal(mf_problem, linear_schedule, taulist[end]);

In [ ]:
df = AdaptiveQuantumAnnealing.energies_and_bitstrings_qaoa(mf_problem)
df[!,"adaptive_agp_probs"] = ones(size(adaptive_probs)[1])
df[!,"adaptive_probs"] = adaptive_probs;
df[!,"linear_probs"] = linear_probs;
df[!,"linear_agp_probs"] = ones(size(adaptive_probs)[1]);
df[!, "row_number"] = 1:2^N

sort!(df, [:energy]);

In [ ]:
df